In [ ]:
from IPython.display import Image, display, Javascript
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.output import eval_js
from base64 import b64decode

# Primeiro, defina a função take_photo() se não estiver definida
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))
    # Decode Base64 data
    binary = b64decode(data.split(',')[1])
    # Save image
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [ ]:
try:
    # Chama a função para tirar a foto
    filename = take_photo()
    print('Foto salva como {}'.format(filename))

    # Carregar a imagem
    img = cv2.imread(filename)

    if img is None:
        print("Erro: Não foi possível carregar a imagem capturada.")
        faces = []
        num_pessoas = 0
    else:
        img_com_deteccao = img.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        if face_cascade.empty():
            print("Erro: Não foi possível carregar o classificador Haar Cascade.")
            faces = []
            num_pessoas = 0
        else:
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            num_pessoas = len(faces)
            print(f"Número de rostos detectados: {num_pessoas}")

            # Perguntar os nomes das pessoas detectadas
            nomes = []
            if num_pessoas > 0:
                print("=== DIGITE OS NOMES DAS PESSOAS === \n")
                for i in range(num_pessoas):
                    nome = input(f"Digite o nome da pessoa {i+1}: ")
                    nomes.append(nome)
                print("Nomes registrados com sucesso!\n")
            else:
                print("Nenhuma pessoa detectada na foto.")

            # Desenhar retângulos, números e nomes na imagem
            for i, (x, y, w, h) in enumerate(faces):
                # Desenha o retângulo verde
                cv2.rectangle(img_com_deteccao, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Adiciona o número da pessoa
                cv2.putText(img_com_deteccao, f"{i+1}", (x, y - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

                # Adiciona o nome da pessoa (se disponível)
                if i < len(nomes) and nomes[i]:
                    # Calcula posição para o nome (abaixo do rosto)
                    nome_y = y + h + 25

                    # Desenha fundo semi-transparente para o nome
                    overlay = img_com_deteccao.copy()
                    text_size = cv2.getTextSize(nomes[i], cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    cv2.rectangle(overlay, (x, nome_y - 20), (x + text_size[0] + 10, nome_y), (0, 0, 0), -1)
                    cv2.addWeighted(overlay, 0.6, img_com_deteccao, 0.4, 0, img_com_deteccao)

                    # Adiciona o nome
                    cv2.putText(img_com_deteccao, nomes[i], (x + 5, nome_y - 5),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

            # Exibir resultado
            img_com_deteccao_rgb = cv2.cvtColor(img_com_deteccao, cv2.COLOR_BGR2RGB)

            plt.figure(figsize=(12, 10))
            plt.imshow(img_com_deteccao_rgb)

            if num_pessoas > 0:
                plt.title(f"Pessoas Detectadas: {num_pessoas}\nNomes: {', '.join(nomes)}", fontsize=14)
            else:
                plt.title("Nenhuma pessoa detectada na foto", fontsize=14)

            plt.axis('off')
            plt.show()

            # Salvar imagem com os nomes
            if num_pessoas > 0:
                nome_arquivo_saida = 'foto_com_nomes.jpg'
                cv2.imwrite(nome_arquivo_saida, img_com_deteccao)
                print(f"Imagem com nomes salva como: {nome_arquivo_saida}")

            # Resumo final
            print("\n=== RESUMO DA DETECÇÃO ===")
            print(f"Total de pessoas detectadas: {num_pessoas}")
            if num_pessoas > 0:
                for i, nome in enumerate(nomes, 1):
                    print(f"Pessoa {i}: {nome}")

except Exception as err:
    print(f"Ocorreu um erro: {str(err)}")
    faces = []
    num_pessoas = 0
    filename = None